# 01_build_public.ipynb（公开可复现构建脚本）

> 目的：把 `data_public/` 里的脱敏数据，统一产出到 `outputs_public/`  
> - 图表：fig1~fig7（供 02_insights_report_public.ipynb 直接引用）  
> - 表格：tab1~tab3（供 02 报告直接展示）  
> - （加分项）SQL：把 public 数据灌入 SQLite 并跑 2-3 条查询，展示“会用 SQL 做分析”的能力

⚠️注意：本 Notebook **只使用脱敏后的 data_public**，不读取 data_raw/data_clean。这样你上传 GitHub 时不会泄露真实门店信息。


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# =========
# 路径约定
# =========
# 推荐项目结构：
# project/
#   data_public/              <- 你刚刚导出的 3 个 csv 放这里
#   outputs_public/           <- 本 notebook 运行后自动生成
#   01_build_public.ipynb
#   02_insights_report_public.ipynb

ROOT = Path.cwd()
DATA_PUBLIC = ROOT / "data_public"
OUT_PUBLIC = ROOT / "outputs_public"
OUT_PUBLIC.mkdir(exist_ok=True)

print("ROOT =", ROOT)
print("DATA_PUBLIC exists:", DATA_PUBLIC.exists(), "|", DATA_PUBLIC)
print("OUT_PUBLIC =", OUT_PUBLIC)


In [ ]:
# =============
# 读取公开数据
# =============
daily = pd.read_csv(DATA_PUBLIC / "daily_public.csv")
store_day = pd.read_csv(DATA_PUBLIC / "store_day_public.csv")
win_store = pd.read_csv(DATA_PUBLIC / "window_store_public.csv")

# 日期字段统一转 datetime
for df in [daily, store_day]:
    df["date"] = pd.to_datetime(df["date"])

print("daily:", daily.shape, daily.columns.tolist())
print("store_day:", store_day.shape, store_day.columns.tolist())
print("window_store:", win_store.shape, win_store.columns.tolist())

display(daily.head(3))
display(store_day.head(3))
display(win_store.head(3))


## 1）定义对比窗口（first7 vs last7）

我们用“**首 7 天** vs **末 7 天**”做对比，是为了把一个月的数据变成一个清晰的问题：

- 如果末 7 天明显比首 7 天差：说明在月内出现了系统性下滑  
- 下滑来自哪里？（订单、SKU、结构、一口价…）→ 后面逐层拆解


In [ ]:
# 取首 7 天、末 7 天（按日期排序）
all_dates = sorted(daily["date"].unique())
first7 = all_dates[:7]
last7 = all_dates[-7:]

print("first7:", [d.date() for d in first7])
print("last7 :", [d.date() for d in last7])

def agg_window(dates):
    sub = store_day[store_day["date"].isin(dates)]
    gmv = sub["gmv"].sum()
    orders = sub["orders"].sum()
    qty = sub["qty"].sum()
    aov = gmv / (orders if orders else np.nan)
    items_per_order = qty / (orders if orders else np.nan)
    return {"gmv": gmv, "orders": orders, "qty": qty, "aov": aov, "items_per_order": items_per_order}

tab1 = pd.DataFrame([
    {"window": "first7", **agg_window(first7)},
    {"window": "last7", **agg_window(last7)},
])

tab1_show = tab1.copy()
tab1_show["gmv"] = tab1_show["gmv"].round(2)
tab1_show["aov"] = tab1_show["aov"].round(2)
tab1_show["items_per_order"] = tab1_show["items_per_order"].round(3)

tab1.to_csv(OUT_PUBLIC / "tab1_window_summary.csv", index=False, encoding="utf-8-sig")
display(tab1_show)
print("✅ saved:", OUT_PUBLIC / "tab1_window_summary.csv")


## 2）趋势图（fig1 / fig2 / fig4 / fig7）

这些图是报告的“叙事骨架”：
- fig1：总 GMV 趋势（是否存在峰值与回落）
- fig2：总 Orders 趋势（GMV 变化主要是否由订单驱动）
- fig4：一口价 GMV 占比趋势（结构变化）
- fig7：Active SKU 趋势（供给侧/上架结构变化）


In [ ]:
# fig1: Total GMV trend
plt.figure(figsize=(10,4))
plt.plot(daily["date"], daily["gmv"])
plt.title("Total GMV by Date (Online Channel)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(OUT_PUBLIC / "fig1_total_gmv_trend.png", dpi=200)
plt.show()

# fig2: Total Orders trend
plt.figure(figsize=(10,4))
plt.plot(daily["date"], daily["orders"])
plt.title("Total Orders by Date (Online Channel)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(OUT_PUBLIC / "fig2_total_orders_trend.png", dpi=200)
plt.show()

# fig4: One-price GMV share trend
plt.figure(figsize=(10,4))
plt.plot(daily["date"], daily["oneprice_share"])
plt.title("One-price GMV Share by Date (Online Channel)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(OUT_PUBLIC / "fig4_oneprice_share_trend.png", dpi=200)
plt.show()

# fig7: Total Active SKU trend
# daily_public 里列名是 active_sku_sum（表示当日各店 active_sku 的加总）
plt.figure(figsize=(10,4))
plt.plot(daily["date"], daily["active_sku_sum"])
plt.title("Total Active SKU by Date (Online Channel)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(OUT_PUBLIC / "fig7_active_sku_trend.png", dpi=200)
plt.show()

print("✅ saved fig1/fig2/fig4/fig7 into", OUT_PUBLIC)


## 3）门店层：谁在拖累整体下滑？（tab2 + fig3）

这一步做的是“贡献拆解”：
- 先找出整体下滑是否集中在少数门店（Pareto）
- 再把“下滑最严重的 Top10 门店”作为重点排查对象


In [ ]:
key = ["store_code", "store_name_public"]

first_agg = (store_day[store_day["date"].isin(first7)]
             .groupby(key, as_index=False)
             .agg(gmv_first7=("gmv","sum"),
                  orders_first7=("orders","sum"),
                  qty_first7=("qty","sum"),
                  oneprice_share_first7=("oneprice_share","mean")))

last_agg = (store_day[store_day["date"].isin(last7)]
            .groupby(key, as_index=False)
            .agg(gmv_last7=("gmv","sum"),
                 orders_last7=("orders","sum"),
                 qty_last7=("qty","sum"),
                 oneprice_share_last7=("oneprice_share","mean")))

tab2 = first_agg.merge(last_agg, on=key, how="outer").fillna(0)

tab2["delta_gmv"] = tab2["gmv_last7"] - tab2["gmv_first7"]
tab2["delta_orders"] = tab2["orders_last7"] - tab2["orders_first7"]
tab2["delta_oneprice_share"] = tab2["oneprice_share_last7"] - tab2["oneprice_share_first7"]

tab2_top10 = tab2.sort_values("delta_gmv").head(10).copy()

tab2_top10.to_csv(OUT_PUBLIC / "tab2_top10_drop_with_oneprice.csv", index=False, encoding="utf-8-sig")
print("✅ saved:", OUT_PUBLIC / "tab2_top10_drop_with_oneprice.csv")
display(tab2_top10)

plt.figure(figsize=(10,4))
plt.barh(tab2_top10["store_name_public"], tab2_top10["delta_gmv"])
plt.title("Top 10 Stores GMV Drop (last7 - first7)")
plt.tight_layout()
plt.savefig(OUT_PUBLIC / "fig3_top10_gmv_drop.png", dpi=200)
plt.show()


## 4）结构线索：一口价占比变化 vs GMV 下滑（fig5 + tab3）

提醒你在报告里要说清楚：

- **相关≠因果**：我们不是在“证明一口价导致 GMV 下滑”，而是在做“线索筛查”
- 如果“占比上升的门店”更容易下滑，就值得在业务上进一步验证


In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(tab2["delta_oneprice_share"], tab2["delta_gmv"])
plt.title("Store-level: delta_oneprice_share vs delta_gmv")
plt.xlabel("delta_oneprice_share (last7 - first7)")
plt.ylabel("delta_gmv (last7 - first7)")
plt.tight_layout()
plt.savefig(OUT_PUBLIC / "fig5_scatter_oneprice_vs_deltagmv.png", dpi=200)
plt.show()

corr = tab2["delta_oneprice_share"].corr(tab2["delta_gmv"])
print("corr =", corr)

tmp2 = tab2.copy()
tmp2["bucket"] = pd.qcut(tmp2["delta_oneprice_share"], 3, labels=["low","mid","high"])

tab3 = (tmp2.groupby("bucket", as_index=False)
        .agg(n=("delta_gmv","size"),
             avg_delta_gmv=("delta_gmv","mean"),
             median_delta_gmv=("delta_gmv","median"),
             avg_delta_oneprice=("delta_oneprice_share","mean")))

tab3.to_csv(OUT_PUBLIC / "tab3_bucket_oneprice_vs_deltagmv.csv", index=False, encoding="utf-8-sig")
print("✅ saved:", OUT_PUBLIC / "tab3_bucket_oneprice_vs_deltagmv.csv")
display(tab3)


## 5）（加分项）SQL：把 public 数据灌入 SQLite，并用 SQL 复现 Top10 下滑

为什么要做这一段？
- JD 常写“SQL 熟练”
- 最好的方式是用 SQL 复现你已经做出来的结论（而不是背语法）


In [ ]:
import sqlite3

DB_PATH = OUT_PUBLIC / "public.sqlite"
conn = sqlite3.connect(DB_PATH)

daily.to_sql("daily_public", conn, if_exists="replace", index=False)
store_day.to_sql("store_day_public", conn, if_exists="replace", index=False)
win_store.to_sql("window_store_public", conn, if_exists="replace", index=False)

print("✅ sqlite saved:", DB_PATH)

first7_str = [d.strftime("%Y-%m-%d") for d in first7]
last7_str  = [d.strftime("%Y-%m-%d") for d in last7]

def sql_in_list(xs):
    return "(" + ",".join([f"'{x}'" for x in xs]) + ")"

sql = f'''
WITH base AS (
  SELECT
    store_code,
    store_name_public,
    CASE
      WHEN date IN {sql_in_list(first7_str)} THEN 'first7'
      WHEN date IN {sql_in_list(last7_str)}  THEN 'last7'
    END AS win,
    SUM(gmv) AS gmv,
    AVG(oneprice_share) AS oneprice_share
  FROM store_day_public
  WHERE date IN {sql_in_list(first7_str)} OR date IN {sql_in_list(last7_str)}
  GROUP BY store_code, store_name_public, win
),
pivot AS (
  SELECT
    store_code,
    store_name_public,
    SUM(CASE WHEN win='first7' THEN gmv ELSE 0 END) AS gmv_first7,
    SUM(CASE WHEN win='last7'  THEN gmv ELSE 0 END) AS gmv_last7,
    MAX(CASE WHEN win='first7' THEN oneprice_share ELSE NULL END) AS oneprice_share_first7,
    MAX(CASE WHEN win='last7'  THEN oneprice_share ELSE NULL END) AS oneprice_share_last7
  FROM base
  GROUP BY store_code, store_name_public
)
SELECT
  store_code,
  store_name_public,
  gmv_first7,
  gmv_last7,
  (gmv_last7 - gmv_first7) AS delta_gmv,
  (COALESCE(oneprice_share_last7,0) - COALESCE(oneprice_share_first7,0)) AS delta_oneprice_share
FROM pivot
ORDER BY delta_gmv ASC
LIMIT 10;
'''

sql_top10 = pd.read_sql_query(sql, conn)
display(sql_top10)

sql_top10.to_csv(OUT_PUBLIC / "tab_sql_top10_drop.csv", index=False, encoding="utf-8-sig")
print("✅ saved:", OUT_PUBLIC / "tab_sql_top10_drop.csv")

conn.close()


## 6）检查输出是否齐全

当你看到 outputs_public/ 里出现这些文件，就说明 01 构建完成，可以去运行 02 报告：

- fig1_total_gmv_trend.png  
- fig2_total_orders_trend.png  
- fig3_top10_gmv_drop.png  
- fig4_oneprice_share_trend.png  
- fig5_scatter_oneprice_vs_deltagmv.png  
- fig7_active_sku_trend.png  
- tab1_window_summary.csv  
- tab2_top10_drop_with_oneprice.csv  
- tab3_bucket_oneprice_vs_deltagmv.csv
